In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('vgsales.csv')

In [3]:
from sqlalchemy import create_engine
from pandas.io import sql
engine = create_engine('sqlite:///:memory:')
df.to_sql('vgsales',engine)

In [ ]:
df.columns

In [ ]:
df['Genre'].unique()

In [4]:
def ARMformat(a,b,tb):
    collect = []
    for x in df[a].unique():
        q = "SELECT "+b+" FROM "+tb+" WHERE "+a+"='"+x+"'"
        res = pd.read_sql_query(q,engine)
        res=res[b].unique()
        resclean = []
        for i in res:
            if i!=None:
                resclean.append(i)
        resmin = resclean[0:10] #terpaksa biar cepat saja
        collect.append(list(resmin))
    return collect

In [5]:
data = ARMformat('Platform','Publisher','vgsales')
len(data)

31

In [6]:
i = 0
pre = df['Platform'].unique()
for dt in data:
    print(pre[i],dt,"\n")
    i+=1

Wii ['Nintendo', 'Ubisoft', 'Sega', '505 Games', 'LucasArts', 'Activision', 'Take-Two Interactive', 'Electronic Arts', 'Warner Bros. Interactive Entertainment', 'Disney Interactive Studios'] 

NES ['Nintendo', 'Palcom', 'Enix Corporation', 'Namco Bandai Games', 'Capcom', 'Konami Digital Entertainment', 'Hudson Soft', 'SquareSoft', 'Ocean', 'BPS'] 

GB ['Nintendo', 'Eidos Interactive', 'Konami Digital Entertainment', 'Electronic Arts', 'Enix Corporation', 'Namco Bandai Games', 'Capcom', 'Activision', 'SquareSoft', 'Imagineer'] 

DS ['Nintendo', '505 Games', 'Sega', 'LucasArts', 'Electronic Arts', 'Activision', 'Disney Interactive Studios', 'Warner Bros. Interactive Entertainment', 'Ubisoft', 'Square Enix'] 

X360 ['Microsoft Game Studios', 'Take-Two Interactive', 'Activision', 'Bethesda Softworks', 'Electronic Arts', 'Ubisoft', 'Warner Bros. Interactive Entertainment', 'Capcom', 'Eidos Interactive', 'MTV Games'] 

PS3 ['Take-Two Interactive', 'Activision', 'Sony Computer Entertainment',

FPGrowth

In [ ]:
!pip install pyfpgrowth

In [7]:
import pyfpgrowth

In [8]:
patterns = pyfpgrowth.find_frequent_patterns(data, 5)
for p,d in patterns.items():
    print(p,":",d)

('LucasArts',) : 5
('LucasArts', 'Sega') : 5
('Microsoft Game Studios',) : 5
('Electronic Arts', 'Microsoft Game Studios') : 5
('Activision', 'Microsoft Game Studios') : 5
('Activision', 'Electronic Arts', 'Microsoft Game Studios') : 5
('Bethesda Softworks',) : 5
('Bethesda Softworks', 'Electronic Arts') : 5
('Activision', 'Bethesda Softworks') : 5
('Activision', 'Bethesda Softworks', 'Electronic Arts') : 5
('Atari',) : 5
('SquareSoft',) : 6
('Namco Bandai Games', 'SquareSoft') : 5
('Sony Computer Entertainment', 'Take-Two Interactive') : 5
('Activision', 'Sony Computer Entertainment', 'Take-Two Interactive') : 5
('Konami Digital Entertainment', 'Sony Computer Entertainment') : 5
('Activision', 'Konami Digital Entertainment', 'Sony Computer Entertainment') : 5
('Electronic Arts', 'Sony Computer Entertainment') : 5
('Electronic Arts', 'Sony Computer Entertainment', 'Take-Two Interactive') : 5
('Activision', 'Electronic Arts', 'Sony Computer Entertainment') : 5
('Activision', 'Electronic

In [9]:
rules = pyfpgrowth.generate_association_rules(patterns, 1.0)
for p,d in rules.items():
    print(p,"=>",d)

('LucasArts',) => (('Sega',), 1.0)
('Microsoft Game Studios',) => (('Activision', 'Electronic Arts'), 1.0)
('Activision', 'Microsoft Game Studios') => (('Electronic Arts',), 1.0)
('Electronic Arts', 'Microsoft Game Studios') => (('Activision',), 1.0)
('Bethesda Softworks',) => (('Activision', 'Electronic Arts'), 1.0)
('Activision', 'Bethesda Softworks') => (('Electronic Arts',), 1.0)
('Bethesda Softworks', 'Electronic Arts') => (('Activision',), 1.0)
('Sony Computer Entertainment', 'Take-Two Interactive') => (('Activision', 'Electronic Arts'), 1.0)
('Konami Digital Entertainment', 'Sony Computer Entertainment') => (('Activision',), 1.0)
('Electronic Arts', 'Sony Computer Entertainment') => (('Activision', 'Take-Two Interactive'), 1.0)
('Activision', 'Electronic Arts', 'Sony Computer Entertainment') => (('Take-Two Interactive',), 1.0)
('Activision', 'Sony Computer Entertainment', 'Take-Two Interactive') => (('Electronic Arts',), 1.0)
('Electronic Arts', 'Sony Computer Entertainment', 'T

Apriori

In [ ]:
!pip install efficient_apriori

In [10]:
from efficient_apriori import apriori

In [13]:
itemsets, rules = apriori(data, min_support=0.2,  min_confidence=0.2)

In [14]:
hasil = []
for x in rules:
    hasil.append([str(x.lhs)+"=>"+str(x.rhs),x.support,x.confidence,x.lift,x.conviction])
    print(str(x.lhs)+"=>"+str(x.rhs),x.support,x.confidence,x.lift,x.conviction,sep="\t")

('Eidos Interactive',)=>('Activision',)	0.22580645161290322	1.0	1.722222222222222	419354838.70967734
('Activision',)=>('Eidos Interactive',)	0.22580645161290322	0.3888888888888889	1.722222222222222	1.2668621680149292
('Electronic Arts',)=>('Activision',)	0.4838709677419355	0.9375	1.6145833333333335	6.709677312000001
('Activision',)=>('Electronic Arts',)	0.4838709677419355	0.8333333333333334	1.6145833333333335	2.903225789032259
('Konami Digital Entertainment',)=>('Activision',)	0.22580645161290322	0.5833333333333334	1.0046296296296295	1.006451610487742
('Activision',)=>('Konami Digital Entertainment',)	0.22580645161290322	0.3888888888888889	1.0046296296296295	1.0029325496784858
('Namco Bandai Games',)=>('Activision',)	0.25806451612903225	0.6153846153846154	1.0598290598290598	1.0903225778103225
('Activision',)=>('Namco Bandai Games',)	0.25806451612903225	0.4444444444444444	1.0598290598290598	1.0451612884412902
('Sega',)=>('Activision',)	0.25806451612903225	0.5714285714285714	0.9841269841